In [ ]:
#### pip install these two libraries every time you start a new server
!pip install xarray
!pip install xarray[io]

In [ ]:
import os
import io
import pathlib
import numpy
import pandas
import seaborn
from pathlib import Path
from matplotlib import pyplot as plt, ticker
import matplotlib
from matplotlib.colors import LogNorm
import datetime
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import multiprocessing
import configparser
import subprocess
import xarray
import itertools
import math
from scipy.constants import k as kB

print(f'You have {int(multiprocessing.cpu_count()-1)} cpu cores available')

In [ ]:
#### Set the file paths for tmp and home directory and MCTrans++ executable here
temp_directory_filepath = '/tmp/Alex_MCTrans'
home_directory_filepath_for_results = '/home/agargone/Alex_MCTrans'
MCTrans_path = '/home/shared/MCTrans/MCTrans/MCTrans++'

In [ ]:
def parallel_MCTrans_multi_variable(df, run_idx):
    df.reset_index(inplace=True)
    config_path = df.at[0,'FilePath']
    
    # if Path(f'{config_path}.config').is_file():
    #     #### USE THIS TO DEBUG MCTRANS RUNS IF THEY ARE CRASHING
    #     print('config file')
    #     config_file_path =  f'{config_path}.config'
    #     !$MCTrans_path $config_file_path
    #     return None

    #### Subprocess runs MCTrans without printing any of the output
    subprocess.run([MCTrans_path, f'{config_path}.config'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    interim_results_df = pandas.DataFrame()
    interim_results_df.at[0,'Run Index'] = run_idx
    interim_results_df.at[0,'Sweep_parameters_name_1'] = df.at[0,'param_1_name']
    interim_results_df.at[0,'Sweep_parameters_name_2'] = df.at[0,'param_2_name']
    interim_results_df.at[0,'Sweep_parameters_name_3'] = df.at[0,'param_3_name']
    interim_results_df.at[0,'Sweep_parameters_name_4'] = df.at[0,'param_4_name']
    interim_results_df.at[0,'Sweep_parameters_name_5'] = df.at[0,'param_5_name']
    interim_results_df.at[0,'Sweep_parameters_name_6'] = df.at[0,'param_6_name']
    interim_results_df.at[0,'Sweep_parameters_value_1'] = df.at[0,'param_1']
    interim_results_df.at[0,'Sweep_parameters_value_2'] = df.at[0,'param_2']
    interim_results_df.at[0,'Sweep_parameters_value_3'] = df.at[0,'param_3']
    interim_results_df.at[0,'Sweep_parameters_value_4'] = df.at[0,'param_4']
    interim_results_df.at[0,'Sweep_parameters_value_5'] = df.at[0,'param_5']
    interim_results_df.at[0,'Sweep_parameters_value_6'] = df.at[0,'param_6']
    interim_results_df.at[0,'FilePath'] = config_path
    #### Build results DataFrame
    if Path(f'{config_path}.nc').is_file():
        # print('FILE')
        data_set = xarray.open_dataset(f'{config_path}.nc')
        data_set_df = data_set.to_dataframe()
    else:
        # print('SKIP')
        data_set_df = pandas.DataFrame()
    interim_results_df = pandas.concat([interim_results_df, data_set_df],axis=1)
    return interim_results_df

In [ ]:
def multi_variable_config_init_worker(_base_config, _MCTrans_path, _temp_directory_filepath, _Study_name, _Study_sweep_variable, _Sweep_parameters_1, _Sweep_parameters_2, _Sweep_parameters_3, _Sweep_parameters_4, _Sweep_parameters_5, _Sweep_parameters_6):
    global base_config, MCTrans_path, temp_directory_filepath, Study_name, Study_sweep_variable, Sweep_parameters_1, Sweep_parameters_2, Sweep_parameters_3, Sweep_parameters_4, Sweep_parameters_5, Sweep_parameters_6
    base_config, MCTrans_path, temp_directory_filepath, Study_name, Study_sweep_variable, Sweep_parameters_1, Sweep_parameters_2, Sweep_parameters_3, Sweep_parameters_4, Sweep_parameters_5, Sweep_parameters_6 = _base_config, _MCTrans_path, _temp_directory_filepath, _Study_name, _Study_sweep_variable, _Sweep_parameters_1, _Sweep_parameters_2, _Sweep_parameters_3, _Sweep_parameters_4, _Sweep_parameters_5, _Sweep_parameters_6
    #### Set environement variables for each parallel process for config file writer

In [ ]:
def multi_variable_config_writer(idx_tuple):
    param_1 = float(Sweep_parameters_1[int(idx_tuple[1][0])])
    param_2 = float(Sweep_parameters_2[int(idx_tuple[1][1])])
    param_3 = float(Sweep_parameters_3[int(idx_tuple[1][2])])
    param_4 = float(Sweep_parameters_4[int(idx_tuple[1][3])])
    param_5 = float(Sweep_parameters_5[int(idx_tuple[1][4])])
    param_6 = float(Sweep_parameters_6[int(idx_tuple[1][5])])
    run_idx_identifier = idx_tuple[0]
    #### Unpack sweep parameters for this specific config file
    
    base_config['configuration'][Sweep_parameters_name_1] = str(param_1)
    base_config['configuration'][Sweep_parameters_name_2] = str(param_2)
    base_config['configuration'][Sweep_parameters_name_3] = str(param_3)
    base_config['configuration'][Sweep_parameters_name_4] = str(param_4)
    base_config['configuration'][Sweep_parameters_name_5] = str(param_5)
    base_config['configuration'][Sweep_parameters_name_6] = str(param_6)
    base_config['configuration']['CentralCellField'] = str(param_2/param_3)    
    base_config['configuration']['PlasmaRadiusMax'] = str((param_4*(0.3*numpy.sqrt(param_3))))
    base_config['configuration']['WallRadius'] = str(0.01+float(base_config['configuration']['PlasmaRadiusMax']))
    #### Modify the base configuration file and replace with sweep parameters => additional inputs that are not sweep parameters can be hardcoded based off sweep parameters
    
    specific_output_filepath = pathlib.Path(output_filepath) / 'configs_and_outputs' / f'{Study_sweep_variable}_{run_idx_identifier}_{str(param_1)}_{str(param_2)}_{str(param_3)}_{str(param_4)}_{str(param_5)}_{str(param_6)}_run'
    specific_output_filepath.mkdir(parents=True, exist_ok=True)
    filepath = f'{specific_output_filepath}/{Study_name}_{Study_sweep_variable}_{run_idx_identifier}_{str(param_1)}_{str(param_2)}_{str(param_3)}_{str(param_4)}_{str(param_5)}_{str(param_6)}_run'
    base_config['algorithm']['AsciiOutputFile'] = f'"{filepath}.out"'
    base_config['algorithm']['NetcdfOutput'] = f'"{filepath}.nc"'
    #### Create the directory for this particluar run and set the output paths
    
    with open(f'{filepath}.config', 'w') as configfile:
        configfile.write('Mode = "SteadyState"\n\n')# Pre-header
        base_config.write(configfile)
    return run_idx_identifier, filepath, param_1, param_2, param_3, param_4, param_5, param_6

In [ ]:
Project_study_name = 'Q_GT1_OBJECTIVE_3_studies'
Study_name = 'Voltage_ThroatField_MirrorRatio_PlasmaRadiusMax_PlasmaLength_ElectronDensity_DT_sweep_study_MRI_fixed_r_exhaust' #_20T_Bmax'
Study_sweep_variable = 'V_TF_MR_PRM_PL_ED'

#### Define the Project, the specific study, and the initials for the variables being swept ==> this is used to build the directory tree

base_config = configparser.ConfigParser()
base_config.optionxform = str  # Preserve case sensitivity

base_config['algorithm'] = {
    'UseAmbipolarPhi': 'true',
    'IncludeChargeExchangeLosses': 'true',
    'AsciiOutputFile': '"{}_{}.out"'.format(Study_name, Study_sweep_variable),
    'NetcdfOutput': '"{}_{}.nc"'.format(Study_name, Study_sweep_variable),
}

base_config['configuration'] = {
    'IonSpecies': '"DT Fuel"',
    # 'InitialMach': '4.0',
    # 'InitialTemp': '0.6',
    'IonToElectronTemperatureRatio': '1.0',
    'Zeff': '1.0',
    'ElectronDensity': '0.1',
    'CentralCellField': '0.45',
    'ThroatField': '4.5',
    # 'ThroatField': '1.5',
    'Voltage': '30000.0',
    'PlasmaRadiusMin': '0.1',
    'ExhaustRadius': '0.1',
    # 'PlasmaRadiusMin': '0.009344',
    # 'ExhaustRadius': '0.009344',
    # 'PlasmaRadiusMin': '0.1287578',
    # 'ExhaustRadius': '0.1287578',
    'PlasmaRadiusMax': '0.055',
    'WallRadius': '0.06',
    'PlasmaLength': '2.141667',
    'AuxiliaryHeating': '0.0',
    'IncludeAlphaHeating': 'true',
    'ReportNuclearDiagnostics': 'true',
    # 'NeutralDensity': '3.218833300054331e-07'
}
#### Define the base config file that will have common parameters between all the runs in the sweep

config_file_paths_df = pandas.DataFrame()

output_filepath = pathlib.Path(temp_directory_filepath) / Project_study_name / Study_name
output_filepath.mkdir(parents=True, exist_ok=True)

output_filepath_for_results = pathlib.Path(home_directory_filepath_for_results) / Project_study_name / Study_name
output_filepath_for_results.mkdir(parents=True, exist_ok=True)

with open(f'{output_filepath}/{Study_name}_base_config_file.config', 'w') as configfile:
    configfile.write('Mode = "SteadyState"\n\n')# Pre-header
    base_config.write(configfile)

with open(f'{output_filepath_for_results}/{Study_name}_base_config_file.config', 'w') as configfile:
    configfile.write('Mode = "SteadyState"\n\n')# Pre-header
    base_config.write(configfile)

#### Save the base config file to both temp and home directory incase you want to rebuild it later

In [ ]:
start_time0 = datetime.datetime.now()

Sweep_parameters_1 = numpy.linspace(500e3, 20e6, num=40)
Sweep_parameters_1 = numpy.round(Sweep_parameters_1,decimals=6)
Sweep_parameters_name_1 = 'Voltage'
len_Swp1 = range(len(Sweep_parameters_1))

Sweep_parameters_2 = numpy.linspace(1.0, 20.0, num=20)
Sweep_parameters_2 = numpy.round(Sweep_parameters_2,decimals=6)
Sweep_parameters_name_2 = 'ThroatField'
len_Swp2 = range(len(Sweep_parameters_2))

Sweep_parameters_3 = numpy.linspace(1.25, 70.0, num=26)
Sweep_parameters_3 = numpy.round(Sweep_parameters_3,decimals=6)
Sweep_parameters_name_3 = 'MirrorRatio'
len_Swp3 = range(len(Sweep_parameters_3))

Sweep_parameters_4 = numpy.linspace(0.1, 1.0, num=10)
Sweep_parameters_4 = numpy.round(Sweep_parameters_4,decimals=6)
Sweep_parameters_name_4 = 'PlasmaRadiusMax'
len_Swp4 = range(len(Sweep_parameters_4))

Sweep_parameters_5 = numpy.append(numpy.linspace(0.1, 1.0, num=4),numpy.linspace(2.5, 20.0, num=8))
Sweep_parameters_name_5 = 'PlasmaLength'
len_Swp5 = range(len(Sweep_parameters_5))

Sweep_parameters_6 = numpy.append(numpy.linspace(0.01, 0.1, num=10),numpy.linspace(0.2, 1.0, num=9))
Sweep_parameters_name_6 = 'ElectronDensity'
len_Swp6 = range(len(Sweep_parameters_6))

#### Define the sweep parameters and their values. Here we are using linspace to create the arrays, but the parameters can have single values defined as so: [0.4]

sweep_combos = enumerate(itertools.product(len_Swp1, len_Swp2, len_Swp3, len_Swp4,len_Swp5,len_Swp6))
total = len(Sweep_parameters_1)*len(Sweep_parameters_2)*len(Sweep_parameters_3)*len(Sweep_parameters_4)*len(Sweep_parameters_5)*len(Sweep_parameters_6)
print(f'total: {total}')
print('total: {:0.9e}'.format(total))
n_workers = int(multiprocessing.cpu_count()-1)
chunksize = max(1, math.floor(total / (n_workers * 300)))
print(f'chunksize: {chunksize}')
#### Enumerate all the combination of variables

filepaths = numpy.empty(total, dtype=object)
col_run_idx = numpy.empty(total, dtype=float)
col_p1 = numpy.empty(total, dtype=float)
col_p2 = numpy.empty(total, dtype=float)
col_p3 = numpy.empty(total, dtype=float)
col_p4 = numpy.empty(total, dtype=float)
col_p5 = numpy.empty(total, dtype=float)
col_p6 = numpy.empty(total, dtype=float)


with ProcessPoolExecutor(max_workers=n_workers, initializer=multi_variable_config_init_worker, initargs=(base_config, MCTrans_path, temp_directory_filepath, Study_name, Study_sweep_variable, Sweep_parameters_1, Sweep_parameters_2, Sweep_parameters_3, Sweep_parameters_4, Sweep_parameters_5, Sweep_parameters_6)) as executor:
    for run_idx, path, p1, p2, p3, p4, p5, p6 in executor.map(multi_variable_config_writer, sweep_combos, chunksize=chunksize):
        filepaths[run_idx] = path
        col_run_idx[run_idx] = run_idx
        col_p1[run_idx] = p1
        col_p2[run_idx] = p2
        col_p3[run_idx] = p3
        col_p4[run_idx] = p4
        col_p5[run_idx] = p5
        col_p6[run_idx] = p6

#### Parallelized config file writer that maps each combination of variables to one processs

config_file_paths_df = pandas.DataFrame()
config_file_paths_df['FilePath'] = filepaths
config_file_paths_df['run_idx'] = col_run_idx
config_file_paths_df['param_1'] = col_p1
config_file_paths_df['param_1_name'] = Sweep_parameters_name_1
config_file_paths_df['param_2'] = col_p2
config_file_paths_df['param_2_name'] = Sweep_parameters_name_2
config_file_paths_df['param_3'] = col_p3
config_file_paths_df['param_3_name'] = Sweep_parameters_name_3
config_file_paths_df['param_4'] = col_p4
config_file_paths_df['param_4_name'] = Sweep_parameters_name_4
config_file_paths_df['param_5'] = col_p5
config_file_paths_df['param_5_name'] = Sweep_parameters_name_5
config_file_paths_df['param_6'] = col_p6
config_file_paths_df['param_6_name'] = Sweep_parameters_name_6
print(config_file_paths_df)
config_file_paths_df.to_csv(f'{output_filepath}/config_file_paths_df.csv',index=False)
config_file_paths_df.to_csv(f'{output_filepath_for_results}/config_file_paths_df.csv',index=False)

#### Save the config file paths and associated sweep values in case you want to rebuild the sweep later

end_time0 = datetime.datetime.now()
print("Done with MCTrans Config File Generator! With execution time: {} s".format(end_time0 - start_time0))

In [ ]:
### READ & RUN MCTRANS DATA ###
###############################
start_time1 = datetime.datetime.now()

if Path(f'{output_filepath}/config_file_paths_df.csv').is_file():
    config_file_paths_df = pandas.read_csv(Path(f'{output_filepath}/config_file_paths_df.csv'))
    #### Check to ensure the config file DataFrame exists

n_workers = min(len(config_file_paths_df), int(multiprocessing.cpu_count()-1))
#### Use all but one of the cpu's on the machine

print(f'Running with {n_workers} workers')

print(f'Length of sweep: {len(config_file_paths_df)}')

futures_list = []
with ProcessPoolExecutor(max_workers=n_workers) as executor:
    futures = {executor.submit(parallel_MCTrans_multi_variable, config_file_paths_df.iloc[[t]], t): t for t in range(len(config_file_paths_df))}
    for future in as_completed(futures):
        try:
            futures_list.append(future)
            # print(f"Finished run {futures[future]}/{len(config_file_paths_df)-1}")
        except Exception as e:
            print(f"Error in iteration {futures[future]}: {e}")
            
#### Run the MCTrans sweep (this may take a while so just wait please
        
end_time1 = datetime.datetime.now()
print("Done with MCTrans Calcs! With execution time: {} s".format(end_time1 - start_time1))

In [ ]:
start_time3 = datetime.datetime.now()
results_df = pandas.DataFrame()

results_df = pandas.concat([future_list.result() for future_list in futures_list])
results_df.sort_values(by='Run Index', inplace=True)
results_df.reset_index(inplace=True,drop=True)
results_df.to_csv(f'{output_filepath}/sweep_results.csv',index=False) 
results_df.to_csv(f'{output_filepath_for_results}/sweep_results.csv',index=False)

#### Aggregate all the results DataFrames into one large DataFrame and save all the results as a csv file to both the temp and home directory. Temp is faster for analysis but home is persistant and saves the data


end_time3 = datetime.datetime.now()
print("Done with MCTrans Result Analysis! With execution time: {} s".format(end_time3 - start_time3))

In [ ]:
print(results_df.columns)
#### Check the results to ensure there is data

In [ ]:
results_df['rho_i_over_R_wall'] = results_df['RhoIon']/results_df['R_wall']
# results_df['neutrons_per_second'] = results_df['DDNeutrons']*100
results_df['neutrons_per_second'] = results_df['FusionNeutronPower']/(14.1e6/6.241509074460762607e18)
results_df['neutrons_per_second_per_Watt'] = results_df['neutrons_per_second']/results_df['RotationPower']
results_df['neutrons_per_second_per_m3'] = results_df['neutrons_per_second']/results_df['V_plasma']
results_df['Mirror_Ratio'] = results_df['B_throat']/results_df['B_midplane']
results_df['Aspect_Ratio'] = (results_df['R_wall']/results_df['L_plasma'])
results_df['Aspect_Ratio_plasma_width'] = (results_df['a']/results_df['L_plasma'])
results_df['Aspect_Ratio_Ms'] = numpy.sqrt((results_df['a']/results_df['L_plasma'])*numpy.log(1e8))
results_df['NeutralDensity_torr'] = (results_df['NeutralDensity']*kB*300)/133.322

results_df.to_csv(f'{output_filepath}/sweep_results.csv',index=False) 
results_df.to_csv(f'{output_filepath_for_results}/sweep_results.csv',index=False)
#### Add additional metrics that we care about to the results DataFrame

In [ ]:
print(results_df)
#### Recheck the results

In [ ]:
#### SYNC ALL THE RESULTS TO YOUR HOME DIRECTORY THIS WILL TAKE A LONG TIME
###############################
source_path = f'{output_filepath}/'
dest_path = f'{output_filepath_for_results}/'
print(dest_path)
print(f'rclone sync -P --multi-thread-streams 1024 {source_path} {dest_path}')
!rclone sync -P --multi-thread-streams 128 $source_path $dest_path

In [ ]:
#### READ TEMP DIRECTORY PRE-ANAYLZED DATA
###############################
if Path(f'{output_filepath}/sweep_results.csv').is_file():
    results_df = pandas.read_csv(f'{output_filepath}/sweep_results.csv')
print(results_df)

In [ ]:
#### READ HOME DIRECTORY PRE-ANAYLZED DATA
###############################
if Path(f'{output_filepath_for_results}/sweep_results.csv').is_file():
    results_df = pandas.read_csv(f'{output_filepath_for_results}/sweep_results.csv')
print(results_df)

In [ ]:
#### FILTER AND REMOVE THE UNPHYSICAL RESULTS ***VERY IMPORTANT
###############################
Alfven_filtered_results_df = results_df.where(results_df['AlfvenMach']<=1.25)
Alfven_filtered_results_df.dropna(inplace=True)
# print(Alfven_filtered_results_df)
print(f'Alfven_filtered_results_df: {len(Alfven_filtered_results_df)}')

RhoStar_filtered_results_df = Alfven_filtered_results_df.where(Alfven_filtered_results_df['RhoStar']<=0.2)
RhoStar_filtered_results_df.dropna(inplace=True)
print(f'RhoStar_filtered_results_df: {len(RhoStar_filtered_results_df)}')
# print(RhoStar_filtered_results_df)

IonCollisionality_filtered_results_df = RhoStar_filtered_results_df.where(RhoStar_filtered_results_df['IonCollisionality']<=1e-1)
IonCollisionality_filtered_results_df.dropna(inplace=True)
print(f'IonCollisionality_filtered_results_df: {len(IonCollisionality_filtered_results_df)}')
# print(IonCollisionality_filtered_results_df)

aspect_ratio_filtered_results_df = IonCollisionality_filtered_results_df.where(IonCollisionality_filtered_results_df['Mach']>IonCollisionality_filtered_results_df['Aspect_Ratio_Ms'])
aspect_ratio_filtered_results_df.dropna(inplace=True)
print(f'aspect_ratio_filtered_results_df: {len(aspect_ratio_filtered_results_df)}')

neutron_1e14_filtered_results_df = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['neutrons_per_second']>=1e14)
neutron_1e14_filtered_results_df.dropna(inplace=True)
print(f'neutron_1e14_filtered_results_df: {len(neutron_1e14_filtered_results_df)}')
print(neutron_1e14_filtered_results_df)

#### SAVE THE FILTERED RESULT DATAFRAMES TO BOTH TEMP AND HOME DIRECTORY
###############################
aspect_ratio_filtered_results_df.to_csv(f'{output_filepath}/filtered_sweep_results.csv',index=False)
neutron_1e14_filtered_results_df.to_csv(f'{output_filepath}/nps_1e14_filtered_sweep_results.csv',index=False) 
aspect_ratio_filtered_results_df.to_csv(f'{output_filepath_for_results}/filtered_sweep_results.csv',index=False) 
neutron_1e14_filtered_results_df.to_csv(f'{output_filepath_for_results}/nps_1e14_filtered_sweep_results.csv',index=False)

In [ ]:
#### Find your top three results and print out the important metrics
###############################

nps1e14 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['neutrons_per_second']==aspect_ratio_filtered_results_df['neutrons_per_second'].max())
nps1e14.dropna(inplace=True)
max_efficiency_idx_df = (nps1e14.where(nps1e14['neutrons_per_second_per_Watt']==nps1e14['neutrons_per_second_per_Watt'].max()))
max_efficiency_idx_df.dropna(inplace=True)
max_efficiency_idx_df.sort_values(by='neutrons_per_second',inplace=True,ascending=False)
max_efficiency_idx_df.reset_index(inplace=True)
max_efficiency_idx = max_efficiency_idx_df.at[0,'index']

max_efficiency_idx_df2 = (aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['neutrons_per_second_per_Watt']==aspect_ratio_filtered_results_df['neutrons_per_second_per_Watt'].max()))
max_efficiency_idx_df2.sort_values(by='RotationPower',inplace=True,ascending=True)
max_efficiency_idx_df2.reset_index(inplace=True)
max_efficiency_idx2 = max_efficiency_idx_df2.at[0,'index']
# print(max_efficiency_idx_df2)

nps5e13 = aspect_ratio_filtered_results_df.copy()
nps5e13.dropna(inplace=True)
max_efficiency_idx_df3 = (nps5e13.where(nps5e13['neutrons_per_second_per_m3']==nps5e13['neutrons_per_second_per_m3'].max()))
max_efficiency_idx_df3.dropna(inplace=True)
max_efficiency_idx_df3.sort_values(by='V_plasma',inplace=True,ascending=True)
max_efficiency_idx_df3.reset_index(inplace=True)
max_efficiency_idx3 = max_efficiency_idx_df3.at[0,'index']

print('Highest Overall Neutron Rate:')
print('------------------------------------------------')
print(f"Run Index: {results_df.at[max_efficiency_idx, 'Run Index']}")
print(f"RotationPower (kWatts): {results_df.at[max_efficiency_idx, 'RotationPower']*1e-3}")
print(f"Voltage (kV): {results_df.at[max_efficiency_idx, 'Voltage']*1e-3}")
print(f"JRadial (mA): {results_df.at[max_efficiency_idx, 'JRadial']*1e3}")
print(f"FusionNeutronPower (kWatts): {results_df.at[max_efficiency_idx, 'FusionNeutronPower']*1e-3}")
print(f"ScientificQ: {results_df.at[max_efficiency_idx, 'ScientificQ']}")
print("Neutrons/s: {:0.5e}".format(results_df.at[max_efficiency_idx, 'neutrons_per_second']))
print("Neutrons/s/Watt: {:0.5e}".format(results_df.at[max_efficiency_idx, 'neutrons_per_second_per_Watt']))
print("Neutrons/s/m3: {:0.5e}".format(results_df.at[max_efficiency_idx, 'neutrons_per_second_per_m3']))
print(f"Aspect_Ratio: {results_df.at[max_efficiency_idx, 'Aspect_Ratio']}")
print(f"R_wall (m): {results_df.at[max_efficiency_idx, 'R_wall']}")
print(f"L_plasma (m): {results_df.at[max_efficiency_idx, 'L_plasma']}")
print(f"V_plasma (m3): {results_df.at[max_efficiency_idx, 'V_plasma']}")
print(f"B_midplane (T): {results_df.at[max_efficiency_idx, 'B_midplane']}")
print(f"B_throat (T): {results_df.at[max_efficiency_idx, 'B_throat']}")
print(f"Mirror Ratio: {results_df.at[max_efficiency_idx, 'B_throat']/results_df.at[max_efficiency_idx, 'B_midplane']}")
print(f"Beta (%): {results_df.at[max_efficiency_idx, 'Beta']}")
print("ElectronDensity: {}/m3".format(results_df.at[max_efficiency_idx, 'ElectronDensity']*1e20))
print("NeutralDensity: {:0.5e}/m3".format(results_df.at[max_efficiency_idx, 'NeutralDensity']))
print("NeutralDensity: {:0.5e} torr@300K".format(results_df.at[max_efficiency_idx, 'NeutralDensity_torr']))
print(f"RhoStar (rho/a): {results_df.at[max_efficiency_idx, 'RhoStar']}")
print(f"RhoIon: {results_df.at[max_efficiency_idx, 'RhoIon']}")
print(f"Mach: {results_df.at[max_efficiency_idx, 'Mach']}")
print(f"Alfven Mach: {results_df.at[max_efficiency_idx, 'AlfvenMach']}")
print(f"IonTemperature: {results_df.at[max_efficiency_idx, 'IonTemperature']}")
print(f"ElectronTemperature: {results_df.at[max_efficiency_idx, 'ElectronTemperature']}")
print(f"TripleProduct: {results_df.at[max_efficiency_idx, 'TripleProduct']}")
print('------------------------------------------------')
print('')
print('------------------------------------------------')
print('Highest Overall Efficiency:')
print('------------------------------------------------')
print(f"Run Index: {results_df.at[max_efficiency_idx2, 'Run Index']}")
print(f"RotationPower (kWatts): {results_df.at[max_efficiency_idx2, 'RotationPower']*1e-3}")
print(f"Voltage (kV): {results_df.at[max_efficiency_idx2, 'Voltage']*1e-3}")
print(f"JRadial (mA): {results_df.at[max_efficiency_idx2, 'JRadial']*1e3}")
print(f"FusionNeutronPower (kWatts): {results_df.at[max_efficiency_idx2, 'FusionNeutronPower']*1e-3}")
print(f"ScientificQ: {results_df.at[max_efficiency_idx2, 'ScientificQ']}")
print("Neutrons/s: {:0.5e}".format(results_df.at[max_efficiency_idx2, 'neutrons_per_second']))
print("Neutrons/s/Watt: {:0.5e}".format(results_df.at[max_efficiency_idx2, 'neutrons_per_second_per_Watt']))
print("Neutrons/s/m3: {:0.5e}".format(results_df.at[max_efficiency_idx2, 'neutrons_per_second_per_m3']))
print(f"Aspect_Ratio: {results_df.at[max_efficiency_idx2, 'Aspect_Ratio']}")
print(f"R_wall (m): {results_df.at[max_efficiency_idx2, 'R_wall']}")
print(f"L_plasma (m): {results_df.at[max_efficiency_idx2, 'L_plasma']}")
print(f"V_plasma (m3): {results_df.at[max_efficiency_idx2, 'V_plasma']}")
print(f"B_midplane (T): {results_df.at[max_efficiency_idx2, 'B_midplane']}")
print(f"B_throat (T): {results_df.at[max_efficiency_idx2, 'B_throat']}")
print(f"Mirror Ratio: {results_df.at[max_efficiency_idx2, 'B_throat']/results_df.at[max_efficiency_idx2, 'B_midplane']}")
print(f"Beta (%): {results_df.at[max_efficiency_idx2, 'Beta']}")
print("ElectronDensity: {}/m3".format(results_df.at[max_efficiency_idx2, 'ElectronDensity']*1e20))
print("NeutralDensity: {:0.5e}/m3".format(results_df.at[max_efficiency_idx2, 'NeutralDensity']))
print("NeutralDensity: {:0.5e} torr@300K".format(results_df.at[max_efficiency_idx2, 'NeutralDensity_torr']))
print(f"RhoStar (rho/a): {results_df.at[max_efficiency_idx2, 'RhoStar']}")
print(f"RhoIon: {results_df.at[max_efficiency_idx2, 'RhoIon']}")
print(f"Mach: {results_df.at[max_efficiency_idx2, 'Mach']}")
print(f"Alfven Mach: {results_df.at[max_efficiency_idx2, 'AlfvenMach']}")
print(f"IonTemperature: {results_df.at[max_efficiency_idx2, 'IonTemperature']}")
print(f"ElectronTemperature: {results_df.at[max_efficiency_idx2, 'ElectronTemperature']}")
print(f"TripleProduct: {results_df.at[max_efficiency_idx2, 'TripleProduct']}")
print('------------------------------------------------')
print('')
print('------------------------------------------------')
print('Most Neutron Dense:')
print('------------------------------------------------')
print(f"Run Index: {results_df.at[max_efficiency_idx3, 'Run Index']}")
print(f"RotationPower (kWatts): {results_df.at[max_efficiency_idx3, 'RotationPower']*1e-3}")
print(f"Voltage (kV): {results_df.at[max_efficiency_idx3, 'Voltage']*1e-3}")
print(f"JRadial (mA): {results_df.at[max_efficiency_idx3, 'JRadial']*1e3}")
print(f"FusionNeutronPower (kWatts): {results_df.at[max_efficiency_idx3, 'FusionNeutronPower']*1e-3}")
print(f"ScientificQ: {results_df.at[max_efficiency_idx3, 'ScientificQ']}")
print("Neutrons/s: {:0.5e}".format(results_df.at[max_efficiency_idx3, 'neutrons_per_second']))
print("Neutrons/s/Watt: {:0.5e}".format(results_df.at[max_efficiency_idx3, 'neutrons_per_second_per_Watt']))
print("Neutrons/s/m3: {:0.5e}".format(results_df.at[max_efficiency_idx3, 'neutrons_per_second_per_m3']))
print(f"Aspect_Ratio: {results_df.at[max_efficiency_idx3, 'Aspect_Ratio']}")
print(f"R_wall (m): {results_df.at[max_efficiency_idx3, 'R_wall']}")
print(f"L_plasma (m): {results_df.at[max_efficiency_idx3, 'L_plasma']}")
print(f"V_plasma (m3): {results_df.at[max_efficiency_idx3, 'V_plasma']}")
print(f"B_midplane (T): {results_df.at[max_efficiency_idx3, 'B_midplane']}")
print(f"B_throat (T): {results_df.at[max_efficiency_idx3, 'B_throat']}")
print(f"Mirror Ratio: {results_df.at[max_efficiency_idx3, 'B_throat']/results_df.at[max_efficiency_idx3, 'B_midplane']}")
print(f"Beta (%): {results_df.at[max_efficiency_idx3, 'Beta']}")
print("ElectronDensity: {}/m3".format(results_df.at[max_efficiency_idx3, 'ElectronDensity']*1e20))
print("NeutralDensity: {:0.5e}/m3".format(results_df.at[max_efficiency_idx3, 'NeutralDensity']))
print("NeutralDensity: {:0.5e} torr@300K".format(results_df.at[max_efficiency_idx3, 'NeutralDensity_torr']))
print(f"RhoStar (rho/a): {results_df.at[max_efficiency_idx3, 'RhoStar']}")
print(f"RhoIon: {results_df.at[max_efficiency_idx3, 'RhoIon']}")
print(f"Mach: {results_df.at[max_efficiency_idx3, 'Mach']}")
print(f"Alfven Mach: {results_df.at[max_efficiency_idx3, 'AlfvenMach']}")
print(f"IonTemperature: {results_df.at[max_efficiency_idx3, 'IonTemperature']}")
print(f"ElectronTemperature: {results_df.at[max_efficiency_idx3, 'ElectronTemperature']}")
print(f"TripleProduct: {results_df.at[max_efficiency_idx3, 'TripleProduct']}")
print('------------------------------------------------')
print(results_df.columns)

print(len(results_df))

In [ ]:
#### Below are some example plotting functions for the data
###############################

In [ ]:
# unique_eden_values = aspect_ratio_filtered_results_df['ElectronDensity'].unique()
unique_eden_values = results_df['Sweep_parameters_value_6'].unique()
disp_unique_eden_values = numpy.round(unique_eden_values,decimals=3)
len_unique_eden_values = len(unique_eden_values)
print(aspect_ratio_filtered_results_df['ElectronDensity'].unique())
print(len_unique_eden_values)

unique_pressure_values = results_df['NeutralDensity_torr'].unique()
unique_pressure_values = unique_pressure_values[~numpy.isnan(unique_pressure_values)]
# unique_pressure_values = unique_pressure_values_df['NeutralDensity_torr'].unique()
print(unique_pressure_values.min())
print(unique_pressure_values.max())

print(aspect_ratio_filtered_results_df["neutrons_per_second"].max())
# print(aspect_ratio_filtered_results_df['Sweep_parameters_value_5'])
# print(unique_Sweep_parameters_5_values)
# del norm_Neutron_per_s
norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=1.0, vmax=aspect_ratio_filtered_results_df["neutrons_per_second"].max())
# norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=aspect_ratio_filtered_results_df["neutrons_per_second"].min(), vmax=aspect_ratio_filtered_results_df["neutrons_per_second"].max())

# eden_df_1 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[0])
# eden_df_1.dropna(inplace=True)
# eden_df_2 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[1])
# eden_df_2.dropna(inplace=True)
# eden_df_3 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[2])
# eden_df_3.dropna(inplace=True)
# eden_df_4 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[3])
# eden_df_4.dropna(inplace=True)


fig, ax = plt.subplots(len_unique_eden_values,1)
fig.set_figheight((6*len_unique_eden_values)+3)

# fig, ax = plt.subplots(4,1)
fig.set_figwidth(16)
# fig.set_figheight(27)
seaborn.set(font_scale=1.0, style='darkgrid')
# fig.suptitle(f'Density: {den_vals}e20 m$^{-3}$')

for iii in range(len_unique_eden_values):
    eden_df = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['Sweep_parameters_value_6']==unique_eden_values[iii])
    eden_df.dropna(inplace=True)
    eden_df_scatter = seaborn.scatterplot(data=eden_df,x='V_plasma',y='Voltage', ax=ax[iii], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

# eden_df_1_scatter = seaborn.scatterplot(data=eden_df_1,x='V_plasma',y='Voltage', ax=ax[0], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_2_scatter = seaborn.scatterplot(data=eden_df_2,x='V_plasma',y='Voltage', ax=ax[1], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_3_scatter = seaborn.scatterplot(data=eden_df_3,x='V_plasma',y='Voltage', ax=ax[2], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_4_scatter = seaborn.scatterplot(data=eden_df_4,x='V_plasma',y='Voltage', ax=ax[3], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

sm = matplotlib.cm.ScalarMappable(cmap="viridis", norm=norm_Neutron_per_s)
sm.set_array([])

for ax_idx in range(len(ax)):
    plt.colorbar(sm, ax=ax[ax_idx], label="Neutron/s")
    ax[ax_idx].set_xlabel('Plasma Volume (m$^3$)')
    ax[ax_idx].set_ylabel('Cathode Voltage (V)')
    ax[ax_idx].set_title(f'Density: {disp_unique_eden_values[ax_idx]}e20 m$^{-3}$')
    # ax[ax_idx].set_ylim([35e3,325e3])
    # ax[ax_idx].set_xlim([-0.001,0.09])
    # ax[ax_idx].set_ylim([150e3,650e3])
    # ax[ax_idx].set_xlim([-1.0,15.0])
    # ax[ax_idx].set_ylim([0,1200e3])
    # ax[ax_idx].set_xlim([-5.0,100.0])
    # ax[ax_idx].set_ylim([0,320e3])
    # ax[ax_idx].set_xlim([-0.0025,0.045])
#     ax[ax_idx].set_ylim([1e3,1e9])
#     ax[ax_idx].set_xlim([unique_pressure_values.min()*0.1,unique_pressure_values.max()])
#     ax[ax_idx].set_xlabel('Neutral Pressure (torr)')
#     ax[ax_idx].set_ylabel('Required Rotation Power (Watts)')


# ax[0].set_xscale('log')
# ax[1].set_xscale('log')
# ax[2].set_xscale('log')
# ax[3].set_xscale('log')

# ax[0].set_yscale('log')
# ax[1].set_yscale('log')
# ax[2].set_yscale('log')
# ax[3].set_yscale('log')

plt.savefig(f'{output_filepath}/results_for_Matt_Neutron_per_second_density_neutrons_per_second.png')
plt.savefig(f'{output_filepath_for_results}/results_for_Matt_Neutron_per_second_density_neutrons_per_second.png')

# IonCollisionality
# V_plasma

In [ ]:
# unique_eden_values = aspect_ratio_filtered_results_df['ElectronDensity'].unique()
unique_eden_values = results_df['Sweep_parameters_value_6'].unique()
disp_unique_eden_values = numpy.round(unique_eden_values,decimals=3)
len_unique_eden_values = len(unique_eden_values)
print(len_unique_eden_values)

unique_pressure_values = results_df['NeutralDensity_torr'].unique()
unique_pressure_values = unique_pressure_values[~numpy.isnan(unique_pressure_values)]
# unique_pressure_values = unique_pressure_values_df['NeutralDensity_torr'].unique()
print(unique_pressure_values.min())
print(unique_pressure_values.max())

print(aspect_ratio_filtered_results_df["neutrons_per_second"].max())
# print(aspect_ratio_filtered_results_df['Sweep_parameters_value_5'])
# print(unique_Sweep_parameters_5_values)
# del norm_Neutron_per_s
norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=1.0, vmax=aspect_ratio_filtered_results_df["neutrons_per_second_per_Watt"].max())
# norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=aspect_ratio_filtered_results_df["neutrons_per_second_per_Watt"].min(), vmax=aspect_ratio_filtered_results_df["neutrons_per_second_per_Watt"].max())

# eden_df_1 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[0])
# eden_df_1.dropna(inplace=True)
# eden_df_2 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[1])
# eden_df_2.dropna(inplace=True)
# eden_df_3 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[2])
# eden_df_3.dropna(inplace=True)
# eden_df_4 = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['ElectronDensity']==unique_eden_values[3])
# eden_df_4.dropna(inplace=True)


fig, ax = plt.subplots(len_unique_eden_values,1)
fig.set_figheight((6*len_unique_eden_values)+3)

# fig, ax = plt.subplots(4,1)
fig.set_figwidth(16)
# fig.set_figheight(27)
seaborn.set(font_scale=1.0, style='darkgrid')
# fig.suptitle(f'Density: {den_vals}e20 m$^{-3}$')

for iii in range(len_unique_eden_values):
    eden_df = aspect_ratio_filtered_results_df.where(aspect_ratio_filtered_results_df['Sweep_parameters_value_6']==unique_eden_values[iii])
    eden_df.dropna(inplace=True)
    eden_df_scatter = seaborn.scatterplot(data=eden_df,x='V_plasma',y='Voltage', ax=ax[iii], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

# eden_df_1_scatter = seaborn.scatterplot(data=eden_df_1,x='V_plasma',y='Voltage', ax=ax[0], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_2_scatter = seaborn.scatterplot(data=eden_df_2,x='V_plasma',y='Voltage', ax=ax[1], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_3_scatter = seaborn.scatterplot(data=eden_df_3,x='V_plasma',y='Voltage', ax=ax[2], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_4_scatter = seaborn.scatterplot(data=eden_df_4,x='V_plasma',y='Voltage', ax=ax[3], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

sm = matplotlib.cm.ScalarMappable(cmap="viridis", norm=norm_Neutron_per_s)
sm.set_array([])

for ax_idx in range(len(ax)):
    plt.colorbar(sm, ax=ax[ax_idx], label="Neutron/s/W")
    ax[ax_idx].set_xlabel('Plasma Volume (m$^3$)')
    ax[ax_idx].set_ylabel('Cathode Voltage (V)')
    ax[ax_idx].set_title(f'Density: {disp_unique_eden_values[ax_idx]}e20 m$^{-3}$')
    # ax[ax_idx].set_ylim([35e3,325e3])
    # ax[ax_idx].set_xlim([-0.001,0.09])
    # ax[ax_idx].set_ylim([150e3,650e3])
    # ax[ax_idx].set_xlim([-1.0,15.0])
    # ax[ax_idx].set_ylim([0,1200e3])
    # ax[ax_idx].set_xlim([-5.0,100.0])
    # ax[ax_idx].set_ylim([0,320e3])
    # ax[ax_idx].set_xlim([-0.0025,0.045])
#     ax[ax_idx].set_ylim([1e3,1e9])
#     ax[ax_idx].set_xlim([unique_pressure_values.min()*0.1,unique_pressure_values.max()])
#     ax[ax_idx].set_xlabel('Neutral Pressure (torr)')
#     ax[ax_idx].set_ylabel('Required Rotation Power (Watts)')

# ax[0].set_xscale('log')
# ax[1].set_xscale('log')
# ax[2].set_xscale('log')
# ax[3].set_xscale('log')

# ax[0].set_yscale('log')
# ax[1].set_yscale('log')
# ax[2].set_yscale('log')
# ax[3].set_yscale('log')

# print(aiiaia)
plt.savefig(f'{output_filepath}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_per_Watt_all.png')
plt.savefig(f'{output_filepath_for_results}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_per_Watt_all.png')

# IonCollisionality
# V_plasma
#ScientificQ
# neutron_1e14_filtered_results_df

In [ ]:
unique_eden_values = results_df['Sweep_parameters_value_6'].unique()
print(f'ElectronDensity unique: {neutron_1e14_filtered_results_df['ElectronDensity'].unique()}')
# unique_eden_values = results_df['ElectronDensity'].unique()
unique_eden_values.sort()
disp_unique_eden_values = numpy.round(unique_eden_values,decimals=3)
len_unique_eden_values = len(unique_eden_values)
print(unique_eden_values)
print(len_unique_eden_values)

print(neutron_1e14_filtered_results_df["neutrons_per_second"].max())
# print(aspect_ratio_filtered_results_df['Sweep_parameters_value_5'])
# print(unique_Sweep_parameters_5_values)
# del norm_Neutron_per_s
norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=neutron_1e14_filtered_results_df["neutrons_per_second_per_Watt"].min(), vmax=neutron_1e14_filtered_results_df["neutrons_per_second_per_Watt"].max())
# norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=aspect_ratio_filtered_results_df["neutrons_per_second"].min(), vmax=aspect_ratio_filtered_results_df["neutrons_per_second"].max())

# eden_df_1 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[0])
# eden_df_1.dropna(inplace=True)
# eden_df_2 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[1])
# eden_df_2.dropna(inplace=True)
# eden_df_3 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[2])
# eden_df_3.dropna(inplace=True)
# eden_df_4 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[3])
# eden_df_4.dropna(inplace=True)



fig, ax = plt.subplots(len_unique_eden_values,1)
fig.set_figheight((6*len_unique_eden_values)+3)

# fig, ax = plt.subplots(4,1)
fig.set_figwidth(16)
# fig.set_figheight(27)
seaborn.set(font_scale=1.0, style='darkgrid')
# fig.suptitle(f'Density: {den_vals}e20 m$^{-3}$')

for iii in range(len_unique_eden_values):
    eden_df = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['Sweep_parameters_value_6']==unique_eden_values[iii])
    # eden_df = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[iii])
    eden_df.dropna(inplace=True)
    eden_df_scatter = seaborn.scatterplot(data=eden_df,x='V_plasma',y='Voltage', ax=ax[iii], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

# eden_df_1_scatter = seaborn.scatterplot(data=eden_df_1,x='V_plasma',y='Voltage', ax=ax[0], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_2_scatter = seaborn.scatterplot(data=eden_df_2,x='V_plasma',y='Voltage', ax=ax[1], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_3_scatter = seaborn.scatterplot(data=eden_df_3,x='V_plasma',y='Voltage', ax=ax[2], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_4_scatter = seaborn.scatterplot(data=eden_df_4,x='V_plasma',y='Voltage', ax=ax[3], hue='neutrons_per_second_per_Watt', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

sm = matplotlib.cm.ScalarMappable(cmap="viridis", norm=norm_Neutron_per_s)
sm.set_array([])

for ax_idx in range(len(ax)):
    plt.colorbar(sm, ax=ax[ax_idx], label="Neutron/s/W")
    ax[ax_idx].set_xlabel('Plasma Volume (m$^3$)')
    ax[ax_idx].set_ylabel('Cathode Voltage (V)')
    ax[ax_idx].set_title(f'Density: {disp_unique_eden_values[ax_idx]}e20 m$^{-3}$')
    # ax[ax_idx].set_ylim([35e3,325e3])
    # ax[ax_idx].set_xlim([-0.001,0.09])
    # ax[ax_idx].set_ylim([150e3,650e3])
    # ax[ax_idx].set_xlim([-1.0,15.0])
    # ax[ax_idx].set_ylim([0,1200e3])
    # ax[ax_idx].set_xlim([-5.0,100.0])
    # ax[ax_idx].set_ylim([0,320e3])
    # ax[ax_idx].set_xlim([-0.0025,0.045])
#     ax[ax_idx].set_ylim([1e3,1e9])
#     ax[ax_idx].set_xlim([unique_pressure_values.min()*0.1,unique_pressure_values.max()])
#     ax[ax_idx].set_xlabel('Neutral Pressure (torr)')
#     ax[ax_idx].set_ylabel('Required Rotation Power (Watts)')

# ax[0].set_xscale('log')
# ax[1].set_xscale('log')
# ax[2].set_xscale('log')
# ax[3].set_xscale('log')

# ax[0].set_yscale('log')
# ax[1].set_yscale('log')
# ax[2].set_yscale('log')
# ax[3].set_yscale('log')


# print(aiiaia)
plt.savefig(f'{output_filepath}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_per_Watt_above_1e13.png')
plt.savefig(f'{output_filepath_for_results}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_per_Watt_above_1e13.png')

# IonCollisionality
# V_plasma
# neutron_1e14_filtered_results_df

In [ ]:
unique_eden_values = results_df['Sweep_parameters_value_6'].unique()
print(f'ElectronDensity unique: {neutron_1e14_filtered_results_df['ElectronDensity'].unique()}')
# unique_eden_values = results_df['ElectronDensity'].unique()
unique_eden_values.sort()
disp_unique_eden_values = numpy.round(unique_eden_values,decimals=3)
len_unique_eden_values = len(unique_eden_values)
print(unique_eden_values)
print(len_unique_eden_values)

print(neutron_1e14_filtered_results_df["neutrons_per_second"].max())
# print(aspect_ratio_filtered_results_df['Sweep_parameters_value_5'])
# print(unique_Sweep_parameters_5_values)
# del norm_Neutron_per_s
norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=neutron_1e14_filtered_results_df["neutrons_per_second"].min(), vmax=neutron_1e14_filtered_results_df["neutrons_per_second"].max())
# norm_Neutron_per_s = matplotlib.colors.LogNorm(vmin=aspect_ratio_filtered_results_df["neutrons_per_second"].min(), vmax=aspect_ratio_filtered_results_df["neutrons_per_second"].max())

# eden_df_1 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[0])
# eden_df_1.dropna(inplace=True)
# eden_df_2 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[1])
# eden_df_2.dropna(inplace=True)
# eden_df_3 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[2])
# eden_df_3.dropna(inplace=True)
# eden_df_4 = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[3])
# eden_df_4.dropna(inplace=True)



fig, ax = plt.subplots(len_unique_eden_values,1)
fig.set_figheight((6*len_unique_eden_values)+3)

# fig, ax = plt.subplots(4,1)
fig.set_figwidth(16)
# fig.set_figheight(27)
seaborn.set(font_scale=1.0, style='darkgrid')
# fig.suptitle(f'Density: {den_vals}e20 m$^{-3}$')

for iii in range(len_unique_eden_values):
    eden_df = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['Sweep_parameters_value_6']==unique_eden_values[iii])
    # eden_df = neutron_1e14_filtered_results_df.where(neutron_1e14_filtered_results_df['ElectronDensity']==unique_eden_values[iii])
    eden_df.dropna(inplace=True)
    eden_df_scatter = seaborn.scatterplot(data=eden_df,x='V_plasma',y='Voltage', ax=ax[iii], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

# eden_df_1_scatter = seaborn.scatterplot(data=eden_df_1,x='V_plasma',y='Voltage', ax=ax[0], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_2_scatter = seaborn.scatterplot(data=eden_df_2,x='V_plasma',y='Voltage', ax=ax[1], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_3_scatter = seaborn.scatterplot(data=eden_df_3,x='V_plasma',y='Voltage', ax=ax[2], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)
# eden_df_4_scatter = seaborn.scatterplot(data=eden_df_4,x='V_plasma',y='Voltage', ax=ax[3], hue='neutrons_per_second', palette='viridis', hue_norm=norm_Neutron_per_s, legend=None)

sm = matplotlib.cm.ScalarMappable(cmap="viridis", norm=norm_Neutron_per_s)
sm.set_array([])

for ax_idx in range(len(ax)):
    plt.colorbar(sm, ax=ax[ax_idx], label="Neutron/s")
    ax[ax_idx].set_xlabel('Plasma Volume (m$^3$)')
    ax[ax_idx].set_ylabel('Cathode Voltage (V)')
    ax[ax_idx].set_title(f'Density: {disp_unique_eden_values[ax_idx]}e20 m$^{-3}$')
    # ax[ax_idx].set_ylim([35e3,325e3])
    # ax[ax_idx].set_xlim([-0.001,0.09])
    # ax[ax_idx].set_ylim([150e3,650e3])
    # ax[ax_idx].set_xlim([-1.0,15.0])
    # ax[ax_idx].set_ylim([0,1200e3])
    # ax[ax_idx].set_xlim([-5.0,100.0])
    # ax[ax_idx].set_ylim([0,320e3])
    # ax[ax_idx].set_xlim([-0.0025,0.045])
#     ax[ax_idx].set_ylim([1e3,1e9])
#     ax[ax_idx].set_xlim([unique_pressure_values.min()*0.1,unique_pressure_values.max()])
#     ax[ax_idx].set_xlabel('Neutral Pressure (torr)')
#     ax[ax_idx].set_ylabel('Required Rotation Power (Watts)')

# ax[0].set_xscale('log')
# ax[1].set_xscale('log')
# ax[2].set_xscale('log')
# ax[3].set_xscale('log')

# ax[0].set_yscale('log')
# ax[1].set_yscale('log')
# ax[2].set_yscale('log')
# ax[3].set_yscale('log')


# print(aiiaia)
plt.savefig(f'{output_filepath}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_above_1e13.png')
plt.savefig(f'{output_filepath_for_results}/results_for_Matt_Neutron_per_second_density_neutrons_per_second_above_1e13.png')

# IonCollisionality
# V_plasma
# neutron_1e14_filtered_results_df